In [2]:
import json
import pandas as pd
import numpy as np
import re
import os
import string
from mkdir import mk_dir
from bert_format import train_qa, test_qa, test_yn, bset

# 构建训练集

In [35]:
# col_ll = [
#     ["样本量","num","How many patients are ivolved in this study?"],
#     ["患者性别","gender","What's gender of the patient?"],
#     ["年龄","age","How old is patient?"],
#     ["证据等级","evi"],
#     ["检测方式","mes"]]
col_ll = [
        ["免疫状态","immu","What's patients' immunity state?"],
    ]

df = pd.read_excel("input/human_label.xlsx",index_col=0)
df = df.loc[df["是否是致病菌"] == "yes",:]

for col_l in col_ll:
    
    col_name = col_l[0]
    col_ab = col_l[1]
    
    # 读取标注信息
    qa = []
    
    for idx in df.index:

        if (df.loc[idx, "%s文本" %col_name] != "none") & (df.loc[idx, "%s" %col_name] != "none"):
            
            if col_ab == "evi":
                "What kind of sample was used to isolated %s?" %df.loc[idx, "物种名称"]
            elif col_ab == "mes":
                que = "Which method was used to diagnose the infection of %s" %df.loc[idx, "物种名称"]
            else:
                que = col_l[2]

            res = re.search(r"\b%s\b"%str(df.loc[idx, "%s文本" % col_name]), df.loc[idx, "Full"].replace("\n",""))
            start = res.start()

            # if col_ab == "num":

                # if (abs(df.loc[idx, "%s_start" % col_ab] - start) > 10) & (len(df.loc[idx, "Full"])>df.loc[idx, "%s_start" % col_ab]):
                #     print(start, df.loc[idx, "%s_start" % col_ab], df.loc[idx, "%s文本" % col_name],df.loc[idx, "Full"].replace("\n","")[start-5:start+15] )
                    
            qa.append(
                train_qa(
                    str(idx), 
                    que,
                    str(df.loc[idx, "%s文本" % col_name ]),
                    start,
                    df.loc[idx, "Full"].replace("\n",""))
            )
    bset(qa, "input/input_%s/" % (col_ab) ,"train_set.json" )

# 构建预测集合

In [ ]:
with open("../Pubmed_Search/search_results/search_results_hanimals.txt", "r") as f:
    a = f.read()
    spec_papers_dic = json.loads(a)
# 加载判断了的病原菌信息
prediction = json.load(open("output/output_pat/total_predictions.json", 'r'))
result = {}
for i in prediction.keys():
    paper_id = i.split(":")[0]
    spec = i.split(":")[1]
    ncbi = "https://pubmed.ncbi.nlm.nih.gov/%s/" % paper_id
    result[i] = [paper_id, spec, prediction[i][1][0], ncbi]
result = pd.DataFrame(result, index = ["ID", "SPECIES", "rate", "link"]).T
prediction_extend = {'Bactology':{}, 'Mycology':{}, 'Virology':{}}

for spec_type in spec_papers_dic:
    
    for k, v in prediction.items():

        spec = k.split(":")[1]
        paper_id = k.split(":")[0]
        
        if v[1][0] > 0.5:

            if spec not in prediction_extend[spec_type]:
            
                prediction_extend[spec_type][spec] = {}
                prediction_extend[spec_type][spec][paper_id] = spec_papers_dic[spec_type][spec][paper_id]
                prediction_extend[spec_type][spec][paper_id]["predition"] = v
            else:
                prediction_extend[spec_type][spec][paper_id] = spec_papers_dic[spec_type][spec][paper_id]
                prediction_extend[spec_type][spec][paper_id]["predition"] = v

with open("output/output_pat/prediction_extend.json", "w") as f:
    json.dump(prediction_extend, f)

In [ ]:
pat = json.load(open("output/prediction_extend.json", "r"))

pneumonia = {"bronchitis","bronchopneumonia","pneumonia","pneumonitis","pneumonic","pleuropneumonia","pleural"}
LungRelated = {"bronchus","bronchial","tracheobronchial","bronchiolar","bronchopulmonary","bronchoalveolar","bronchoscopy","lung","pulmonary","pneumonic","bronchopulmonary","respiratory","respiration"}
infection = {"infection","infections","abscess","pathology","etiology","diseases","bacteremia","sepsis","septic","pathogenicity","legionellosis","empyema","necrosis"}

pats_dic = {'Bactology':{}, 'Mycology':{}, 'Virology':{}}
pats = {}

for spec_type in pat:
    
    for spec in pat[spec_type]:
        
        vbp = {}
        for paper_id, paper in pat[spec_type][spec].items():
            
            text = paper["texts"]["Ab"] +" "+" ".join(sum(paper["Meshhead"],[])) # 添加mesh词汇
            text = text.translate(str.maketrans(string.punctuation, " "*32,)).replace("  ", " ")
            text = text.lower()
            words = set(text.split(" "))
            
            if len(words & pneumonia) > 0:
                # if "colonization" not in words:
                #     if "%s / isolation & purification" % spec not in " / ".join(sum(paper["Meshhead"],[])).lower():
                vbp[paper_id+":"+spec] = paper
                
            else:
                if len(words & LungRelated) > 0:
                    if len(words & infection) > 0:
                        # if "colonization" not in words:
                        #     if "%s / isolation & purification" % spec not in " / ".join(sum(paper["Meshhead"],[])).lower():
                        vbp[paper_id+":"+spec] = paper
                        
        if len(vbp)>0:
            pats_dic[spec_type][spec] = vbp
            pats.update(vbp)
with open("output/prediction_p_extend.json","w") as f:
    json.dump(pats_dic,f)

In [3]:
with open("output/output_pat/prediction_extend.json", "r") as f:
    spec_papers_dic = json.load(f)

# 根据上述筛选出的样本的数据集
col_ll = [
    ["样本量","num","How many patients are ivolved in this study?"],
    ["患者性别","gender","What's gender of the patient?"],
    ["年龄","age","How old is patient?"],
    ["免疫状态","immu","What's patients' immunity state?"],
    ["证据等级","evi"],
    ["检测方式","mes"]]

for col_l in col_ll:
    
    col_name = col_l[0]
    col_ab = col_l[1]
    
    qa = []

    for spec_type in spec_papers_dic:
            
        for spec in spec_papers_dic[spec_type]:
            
            for paper_id in spec_papers_dic[spec_type][spec]:
                
                if len(spec_papers_dic[spec_type][spec][paper_id]["texts"]["Ab"]) > 0:
                    
                    if col_ab == "evi":
                        que = "What kind of sample was used to isolated %s?" % spec
                    elif col_ab == "mes":
                        que = "Which method was used to diagnose the infection of %s" % spec
                    else:
                        que = col_l[2]
                    
                    qa.append(test_qa(
                        spec,
                        que,
                        spec_papers_dic[spec_type][spec][paper_id]["texts"]["Ab"]+" "+
                        " ".join(sum(spec_papers_dic[spec_type][spec][paper_id]["Meshhead"],[])) ))
                
    print(len(qa))
    qasub = [qa[i:i + 50000] for i in range(0, len(qa), 50000)]
    # for i in range(len(qasub)):
        
        # bset(qasub[i],"input/input_%s/input_%s/" % (col_ab,i),"test_set.json")
        
        # with open("input/input_%s/train_set.json" % col_ab, "r") as f:
        #     train = json.load(f)
        # with open("input/input_%s/input_%s/train_set.json" % (col_ab,i), "w") as f:
        #     json.dump(train, f)

292821
292821
292821
292821
292821
292821
